# Ch6 - 資料載入、儲存和檔案格式

In [19]:
import pandas as pd
import numpy as np
import json
from lxml import objectify
from io import StringIO
import requests
import sqlite3
import sys
import sqlalchemy as sqla
import psycopg2

In [23]:
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


## pandas中的解析函式
|函式|描述|
|:-:|:-:|
|read_csv|從檔案、URL或類檔案物件中讀取符號分隔資料，預設使用逗號分隔|
|read_table|從檔案、URL或類檔案物件中讀取符號分隔資料，預設使用Tab('\t')分隔|
|read_fwf|從固定寬度欄位讀取資料(例如:沒有分隔符號)|
|read_clipboard|類似read_table，用來從剪貼簿讀取資料，在轉換網頁資料為表格時很好用|
|read_excel|從 Excel XLS 或 XLSX 讀取表格式資料|
|read_hdf|讀取用pandas寫出的HDF5檔案|
|read_html|從指定的HTML文件中讀取所有的表格|
|read_json|從JSON(JavaScript Object notation)字串格式讀取資料|
|read_msgpack|讀取以MessagePack二進位格式編碼過的pandas資料|
|read_pickle|讀取以Python pickle格式所儲存的資料|
|read_sas|從SAS系統的儲存格式中讀取一個SAS資料集|
|read_sql|把一個SQL query(使用SQLAlchemy)的結果，讀成一個pandas DataFame|
|read_stata|從Stata檔案格式讀取資料集|
|read_feather|讀取Feather二進位格式|

* 上述函式都是將文字資料轉換為DataFrame，這些函式的可選參數，可大致分為以下幾種:
    * 索引(indexing):
        * 可以擇一或多個欄位作為DataFrame回傳，還有要不要從檔案中取得欄名稱，使用者名稱或是都不取得
<br><br>
    * 型態推斷與資料轉換:
        * 包括使用者定義值轉換，以其缺失值要用什麼取代
<br><br>
    * 解析日期時間格式:
        * 具組合功能，包括將跨多個欄位的日期和時間資料整合到單一欄中
<br><br>
    * 疊代(iterating):
        * 處理非常大的檔案時，支援多個區塊的疊代運作
<br><br>
    * 未清理的資料問題:
        * 跳過列或是註腳，註解或其他的小東西，例如數值格式資料標示一千的逗號

e.g. CSV文字檔

In [16]:
df = pd.read_csv(r'D:\Python\Python 資料分析\ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 使用read_table

In [18]:
pd.read_table(r'D:\Python\Python 資料分析\ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 讓pandas預設欄名稱

In [20]:
pd.read_csv(r'D:\Python\Python 資料分析\ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 自行指定欄名稱(使用names=[ ]參數)

In [23]:
pd.read_csv(r'D:\Python\Python 資料分析\ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 假設想讓message變成回傳DataFrame的index的話，可以藉由index_col參數指定index為4(message的index)或指定名稱'message'

In [24]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv(r'D:\Python\Python 資料分析\ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


* 假設想依多個欄的值做出階層式index -> 將要用的欄編號或名稱以list傳入

In [25]:
parsed = pd.read_csv(r'D:\Python\Python 資料分析\csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

* 某些情況下，資料表可能沒有固定的分隔符號，只用空白或其他東西做分隔

In [30]:
with open(r'D:\Python\Python 資料分析\ex3.txt') as f:
    print(list(f))

['            A         B         C\n', 'aaa -0.264438 -1.026059 -0.619500\n', 'bbb  0.927272  0.302904 -0.032399\n', 'ccc -0.264273 -0.386314 -0.217601\n', 'ddd -0.871858 -0.348382  1.100491']


* \s表示匹配任何空白字符，包括空格、制表符、换頁符等等,等價於[ \f\n\r\t\v]
* +在正则表达式中表示“匹配一次或多次”
* \s+则表示匹配任意多个上面的字符

In [31]:
result = pd.read_table(r'D:\Python\Python 資料分析\ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


* 上述範例中，因為欄位名稱那列的數量比資料列少一個，因此read_table推斷第一欄為DataFrame的索引

* 解析函式提供許多參數，可以處理可能會碰到的各種檔案格式的異常

In [41]:
with open(r'D:\Python\Python 資料分析\ex4.csv') as f:
    print(list(f))

['# 我要被跳過了,,,,\n', 'a,b,c,d,message\n', '# 包子,,,,\n', '# 一堆包子,,,,\n', '1,2,3,4,hello\n', '5,6,7,8,world\n', '9,10,11,12,foo\n']


* skirows可以用來跳過指定列

In [42]:
pd.read_csv(r'D:\Python\Python 資料分析\ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* pandas會預設使用常用的標記值(NA或NULL)來當作缺失值的標記

In [10]:
result = pd.read_csv(r'D:\Python\Python 資料分析\ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [11]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


* na_values參數設定為一組字串，資料值符合字串者會被當作缺失值

In [12]:
result = pd.read_csv(r'D:\Python\Python 資料分析\ex5.csv', na_values=['world'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,NaN
2,three,9,10,11.0,12,foo


* 使用dict格式可為每個欄指定不同的NA標記

In [13]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv(r'D:\Python\Python 資料分析\ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 部分read_csv/read_table函式參數
|參數|描述|
|-:|:-:|
|path|表示檔案系統，URL或類檔案物件的字串|
|sep 或 delimiter|用來分隔每列欄位的字元序列或正規表達式|
|header|用來當作欄名的列號，預設為0(第一列)，如果沒有欄名列的話，應該要指定為None|
|index_col|指定用來當作列index的欄編號或名稱，可以指定單一欄或是為階層式index指定多個欄|
|names|和header=None合併使用，以list格式指定欄位名稱|
|skiprows|從檔案開始起算，要跳過的列號(首列為0)|
|na_values|要用NA取代掉的值|
|comment|見到指定字元，即認為式註解(一直到行尾)|
|parse_dates|嘗試將資料解析為datetime，預設為False。如果設定為True的話，會試圖把所有欄都解析為datatime，不然可以指定要解析的欄編號或名稱。如果傳遞的是list或tuple格式的話，將會合併多個欄的資料並解析成日期(例如: 日期/時間分別放在兩欄的情況)|
|keep_date_col|如果合併欄位進行日期解析的話，保留原始欄位，預設為False|
|converters|一個含欄編號或名稱，對應到函式的dict型態(例如: {'foo': f})，指定套用f函式於'foo'欄中的所有值|
|dayfirsts|當解析不好分辨日期時，套用國際格式(例如: 7/6/2012 -> Jun 7, 2012)，預設為False|
|date_parser|用來解析日期的格式|
|nrows|從檔頭開始算，要讀幾行|
|iterator|為分段讀取檔案，回傳一個TextParser物件|
|chunksize|疊代時使用，指定每段大小|
|skip_footer|跳過檔案的最後幾行|
|verbose|印出詳細的解析輸出訊息，例如在非數值欄為中有多少缺失值|
|encoding|Unicode的文字編碼(例如: 'utf-8'代表UTF-8編碼文字)|
|squeeze|如果解析的資料只有一個欄位，就回傳一個Series|
|thousands|千分位分隔符號(例如: ',' 或 '.')|

## CSV dialect選項
|參數|描述|
|-:|:-:|
|delimiter|分隔欄位的一個字元字串，預設是','|
|lineterminator|寫出時的行尾標示符號，預設是'\r\n'。讀取時不需使用，會自動識別跨平台行尾符號|
|quotechar|指定欄位的括號標記字元(用法類似delimiter)，預設為'"'|
|quoting|括號原則。可用選項包括csv.QUOTE_ALL(所有欄位都要括)、csv_QUOTE_MINIMAL(只有標記特別字元的欄位才括)、csv.QUOTE_NONNUMERIC以及csv.QUOTE_NONE(不括)，Python文件有完整說明，預設為QUOTE_NONE|
|skipinitialspace|忽略分隔字元後的空白，預設為False|
|doublequote|一個欄位中如何處理括號字元的方法，True為雙倍(查看文檔較清楚)|
|escapechar|如果將quoting設為csv.QUOTE_NONE時，用來脫逸分隔符號的字串，預設是關閉|

## JSON 資料

* JSON(JavaScript Object Notation)變成網路瀏覽器及其他應用程式，透過HTTP傳送資料的標準格式之一。它比像CSV這種表格式文字更有彈性

In [5]:
obj = """

{"name": "Wes",
 "place_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}

"""


* 把JSON轉成Python格式 -> json函式庫

In [6]:
result = json.loads(obj)
result

{'name': 'Wes',
 'place_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

* json.dumps -> 將Python物件轉回JSON

In [7]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "place_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

* 可以將dict的list傳給DataFrame建構子，同時指定選用哪些資料欄位

In [8]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [13]:
example = """

[{"a": 1, "b": 2, "c": 3},
{"a": 4, "b": 5, "c": 6},
{"a": 7, "b": 8, "c": 9}]
"""

* pandas.read_json -> 預設將JSON陣列中每一個物件轉為表格中的一列

In [14]:
data = pd.read_json(example)
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


* to_json -> Series 和 DataFrame都有的方法，可將pandas資料會出道JSON

In [16]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [19]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML和HTML: 從網站抓資料

* pandas.read_html函式 -> 利用項lxml和Beautiful Soup這種函式庫自動解析HTML檔案中的資料，轉換為DataFrame
* paddas.read_html預設會搜尋並企圖解析被\<table>標籤包含的表格式資料，出來的結果會是DataFrame物件的list

In [3]:
tables = pd.read_html(r"https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
len(tables)

1

In [5]:
failures = tables[0]
failures.head()

,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Almena State Bank,Almena,KS,15426,Equity Bank,"October 23, 2020",10538
1,First City Bank of Florida,Fort Walton Beach,FL,16748,"United Fidelity Bank, fsb","October 16, 2020",10537
2,The First State Bank,Barboursville,WV,14361,"MVB Bank, Inc.","April 3, 2020",10536
3,Ericson State Bank,Ericson,NE,18265,Farmers and Merchants Bank,"February 14, 2020",10535
4,City National Bank of New Jersey,Newark,NJ,21111,Industrial Bank,"November 1, 2019",10534


In [6]:
close_timestamps = pd.to_datetime(failures['Closing DateClosing'])
close_timestamps

0     2020-10-23
1     2020-10-16
2     2020-04-03
3     2020-02-14
4     2019-11-01
         ...    
558   2001-07-27
559   2001-05-03
560   2001-02-02
561   2000-12-14
562   2000-10-13
Name: Closing DateClosing, Length: 563, dtype: datetime64[ns]

In [7]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2017      8
2015      8
2016      5
2004      4
2001      4
2020      4
2019      4
2003      3
2007      3
2000      2
Name: Closing DateClosing, dtype: int64

## 用lxml.objectify 解析 XML

* XML(eXtensible Markup Language)是另外一種常用來支援階層式，巢式以及帶metadata的資料結構
* pandas.read_html -> 由lxml或Beautiful Soup對HTML做解析
* XML與HTML結構相似，但XML使用更為普遍

* 使用lxml.objectify可以解析檔案
* 使用getroot取得XML檔案中的根節點參照

In [23]:
path = r'D:\Python\Python 資料分析\Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

* root.INDICATOR回傳一個產生器，會產生出\<INDICATOR> XML元素
* 對每筆紀錄，可以產出標籤名稱與填充資料的一個dict

In [24]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

* 再將dict組成的list轉為DataFrame

In [25]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,12,Service Indicators,M,%,97.0,,97.0,


* HTML連結標籤也是個合法的XML

In [26]:
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

* 現在可以任意存取標籤中的欄位(href)或連結文字了

In [27]:
root

<Element a at 0x25068ff1fc0>

In [28]:
root.get('href')

'http://www.google.com'

In [29]:
root.text

'Google'

## 6.2二進位資料格式

* pandas物件有to_pickle方法，用來將資料以pickle格式寫入到磁碟

In [2]:
frame = pd.read_csv(r'D:\Python\Python 資料分析\ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
frame.to_pickle(r'D:\Python\Python 資料分析\frame_pickle')

* 可以使用pickle讀取檔案中任何被pickle過的物件，或是使用pandas.read_pickle

In [4]:
pd.read_pickle(r'D:\Python\Python 資料分析\frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## HDF5格式

* HDF -> 階層式資料格式(hierarchical data format)
* 每個HDF5檔案可以儲存多個資料集，並支援matadata
* 利用多種壓縮模式支援即時壓縮，使有重複特徵的資料可以用更有效率的方式儲存
* 由於可以有效率的存取大型陣列中的小區塊，所以在處理體積大到放不進記憶體的資料集時，可使用HDF5

In [5]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [7]:
store = pd.HDFStore('mydata.h5')

In [8]:
store['obj1'] = frame

In [9]:
store['obj1_col'] = frame['a']

In [10]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

* 在HDF5檔案中的物件，可以使用dict的API存取

In [11]:
store['obj1']

,a
0,0.085537
1,0.570470
2,1.008683
3,2.539274
4,-0.765881
...,...
95,-1.059704
96,-1.943282
97,-0.436749
98,-1.407617


* HDFStore支援'fixed'和'table'兩種儲存方法
    * 通常table速度較fixed稍慢，但是支援特殊查詢語法

In [13]:
store.put('obj2', frame, format='table')

In [17]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.378738
11,-0.459774
12,0.981579
13,-1.049083
14,-0.915585
15,-0.111969


In [20]:
store.close()

* put是store['obj2'] = frame的顯式寫法，可以讓我們指定像是儲存格式等參數
* pandas.read_hdf函式可以把上面的工具再精簡化

In [21]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [22]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,0.085537
1,0.570470
2,1.008683
3,2.539274
4,-0.765881


## 讀取Microsoft Excel檔案

* 可以使用ExcelFile類別或是pandas.read_excel函式 -> 需額外安裝xlrd和openpyxl套件來讀XLS或XLSX檔案

* 資料解析

In [2]:
xlsx = pd.ExcelFile(r'D:\Python\Python 資料分析\house.xlsx')

* 解析完畢後，就可轉換為DataFrame

In [5]:
house = pd.read_excel(xlsx)
house2 = house.iloc[0:5]
house2

,年度別,合計戶數,應稅戶數,免稅戶數,應稅面積,免稅面積,應稅現值,免稅現值,合計稅額,本稅稅額
0,110年,237727,199834,37893,37043774,10259563,112053363,28997024,1752798,1752798
1,109年,232060,195019,37041,36533026,9977304,108194579,27151073,1733419,1733419
2,108年,228277,192169,36108,36374968,9894492,106022545,26075841,1701839,1701839
3,107年,225274,190267,35007,36085966,9622236,105085031,24550748,1680934,1680934
4,106年,222833,188459,34374,35824085,9353304,103792296,23395525,1642948,1642948


* 使用pandas將資料以excel寫出，須先建立ExcelWriter，然後使用pandas物件的to_excel方法把資料寫給ExcelWriter

In [6]:
writer = pd.ExcelWriter(r'D:\Python\Python 資料分析\house2.xlsx')

In [7]:
house2.to_excel(writer)

In [8]:
writer.save()
writer.close()

D:\conda\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


* 也可以直接把路徑傳給to_excel

In [9]:
house2.to_excel(r'D:\Python\Python 資料分析\house2.xlsx')

## 6.3使用Web API

* 許多網站透過JSON或其他格式，提供API資料點，在Python中有數種方法可以存取這些API
* 這裡使用requests

* e.g. 在GitHub上查詢pandas最新的30個issue -> 可利用requests函式庫做出 GET HTTP request 進行查詢

In [13]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
url

'https://api.github.com/repos/pandas-dev/pandas/issues'

In [14]:
resp = requests.get(url)
resp

<Response [200]>

* get傳回的物件中，有一個json方法，可將JSON轉為python原生物件，並使用字典型態回傳

In [15]:
data = resp.json()
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073/events',
  'html_url': 'https://github.com/pandas-dev/pandas/pull/43073',
  'id': 972215270,
  'node_id': 'MDExOlB1bGxSZXF1ZXN0NzEzODQzNzky',
  'number': 43073,
  'title': 'PERF: is_string_dtype checks',
  'user': {'login': 'jbrockmendel',
   'id': 8078968,
   'node_id': 'MDQ6VXNlcjgwNzg5Njg=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/8078968?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/jbrockmendel',
   'html_url': 'https://github.com/jbrockmendel',
   'followers_url': 'https://api.github.com/users/jbrockmendel/followers',
   'following_url': 'https:/

In [16]:
data[0]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073',
 'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
 'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073/comments',
 'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/43073/events',
 'html_url': 'https://github.com/pandas-dev/pandas/pull/43073',
 'id': 972215270,
 'node_id': 'MDExOlB1bGxSZXF1ZXN0NzEzODQzNzky',
 'number': 43073,
 'title': 'PERF: is_string_dtype checks',
 'user': {'login': 'jbrockmendel',
  'id': 8078968,
  'node_id': 'MDQ6VXNlcjgwNzg5Njg=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/8078968?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/jbrockmendel',
  'html_url': 'https://github.com/jbrockmendel',
  'followers_url': 'https://api.github.com/users/jbrockmendel/followers',
  'following_url': 'https://api.github.com/use

In [17]:
data[0]['title']

'PERF: is_string_dtype checks'

* 在data中的每個元素皆為字典型態，包含了GitHub issue頁面上的所有資料(除了註解)
* 可將data直接傳給DataFrame並指定要用的欄位

In [19]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues.head()

,number,title,labels,state
0,43073,PERF: is_string_dtype checks,[],open
1,43072,ENH: Add Arrow CSV Reader,[],open
2,43071,BENCH: Add pyarrow to env,[],open
3,43070,Backport PR #43033 on branch 1.3.x (DOC: argum...,"[{'id': 1728592794, 'node_id': 'MDU6TGFiZWwxNz...",open
4,43069,Backport PR #43052 on branch 1.3.x (REGR: wind...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open


## 6.4使用資料庫

* 使用sqlite3建立一個SQL Lite資料庫

In [7]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);
"""

In [8]:
con = sqlite3.connect('mydata.sqlite')

In [9]:
con.execute(query)

In [10]:
con.commit()

In [11]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [12]:
stmt = 'INSERT INTO TEST VALUES(?, ?, ?, ?)'

In [13]:
con.executemany(stmt, data)

In [14]:
con.commit()

In [15]:
cursor = con.execute('select * from test')
cursor

In [16]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

* 將此tuple所組成的list傳給DataFrame建構子，再傳入欄位名稱
* 欄位名稱被放在cursor的description 屬性中

In [17]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [18]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


* 因為上述程序繁瑣，因此可以使用python 的 SQLAlchemy專案
* pandas中有一個read_sql函式，可以從SQLAlchemy連線中讀取資料

In [21]:
db = sqla.create_engine('sqlite:///mydata.sqlite')
db

Engine(sqlite:///mydata.sqlite)

In [22]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
